# 2024C3 SMI WAXS TReXS processing notebook
This notebook is for processing tender resonant X-ray scattering data collected on the WAXS detector from SMI. 

**Copy this notebook (along with all other notebooks in this folder) to your own user folder, don't change this one.**

## Imports (ignore the warnings, as long as it runs the cell)

In [ ]:
import pathlib
import os
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from PIL import Image
from tqdm.auto import tqdm 

import fabio
from smi_analysis import SMI_beamline

## Define paths & functions

In [ ]:
propPath = pathlib.Path('/nsls2/data/smi/proposals/2024-3/pass-316856')

rawPaths = propPath.joinpath('raw_05')
saxsPath = rawPaths.joinpath('1M')
waxsPath = rawPaths.joinpath('900KW')

### Custom SMI function
This wraps the smi_analysis loading code to return lists of data/metadata for each measurement

In [ ]:
def waxs_SMI_numpy_loading_wa7wa27(path, filename_sublists):
    """
    Function adapted from Guillames SMI notebooks to process raw waxs tiffs:
    
    Inputs:
        path (pathlib.Path): 
        filename_sublists (list): 
    
    Returns: 
        filename_wa0_list (list): list of filenames loaded
        recip_list (list): list of loaded numpy data arrays (in qx vs qy space)
        recip_extents (list): list of qx qy extents: [qx_min, qx_max, qy_min, qy_max]
        caked_list (list): list of loaded numpy data arrays (in q vs chi space)
        caked_extents (list): list of q chi extents: [q_min, q_max, chi_min, chi_max]
    """
    
    #waxs
    geometry = 'Transmission'
    bs_kind = 'pindiode'
    incident_angle = np.deg2rad(0)

    #WAXS
    detector_waxs = 'Pilatus900kw'
    sdd_waxs = 280
    center_waxs = [97, 1255.9]
    bs_pos_waxs = [[115, 1180], [0, 0], [0, 0]]  # updated 97 to 115, 1050 to 1180 for beamstop at 7 degree waxs arc angle
    
    flatPath = pathlib.Path('/nsls2/data/smi/legacy/results/data/2024_3/314483_Freychet_08/900KW')    
    flatfield = np.rot90(fabio.open(flatPath.joinpath('GF_GF_flatfield_Sedge_2450uhighg1600_Y2_06_2477.00eV_wa20deg_id807229_000000_WAXS.tif')).data, 1)
    
    filename_wa0_list = []
    recip_list = []
    recip_extents = []
    caked_list = []
    caked_extents = []
    for dat in tqdm(filename_sublists, desc='Processing tiffs'):
            # Set waxs detector angles (0.06 degree offset)
            waxs_angle = [np.deg2rad(7-0.06), np.deg2rad(27-0.06)]
            
            # Extract & set energy from filename
            idx = dat[0].find('eV')
            energy = 0.001*float(dat[0][idx-7:idx])
            wav = 1E-10 * (12.398/energy)

            # Define SMI loader object
            SMI_waxs = SMI_beamline.SMI_geometry(geometry = geometry,
                                                 detector = detector_waxs,
                                                 sdd = sdd_waxs,
                                                 wav = wav,
                                                 alphai = 0,
                                                 center = center_waxs,
                                                 bs_pos = bs_pos_waxs,
                                                 det_angles = waxs_angle,
                                                 bs_kind = None)

            # Load data, be aware of load order
            SMI_waxs.open_data(path, dat[::-1], optional_mask='tender')

            # Masking code from Guillame
            SMI_waxs.masks[1][1180:, 100:108] = True
            SMI_waxs.masks[1][1254, 47], SMI_waxs.masks[1][1255, 47]=True, True
            SMI_waxs.masks[4][1254, 47], SMI_waxs.masks[4][1255, 47]=True, True
            SMI_waxs.masks[1][979:1050, 0:100], SMI_waxs.masks[4][979:1050, 0:100]=True, True
            SMI_waxs.masks[1][967, 67], SMI_waxs.masks[4][967, 67]=True, True
            SMI_waxs.masks[1][490:555, 100:], SMI_waxs.masks[4][490:555, 100:]=True, True
            SMI_waxs.masks[1][1231, 174], SMI_waxs.masks[1][1232, 174]=True, True
            SMI_waxs.masks[4][1231, 174], SMI_waxs.masks[4][1232, 174]=True, True
            SMI_waxs.masks[1][1414, 179], SMI_waxs.masks[1][1415, 179]=True, True
            SMI_waxs.masks[4][1414, 179], SMI_waxs.masks[4][1415, 179]=True, True
            SMI_waxs.masks[1][858, 5], SMI_waxs.masks[1][859, 5]=True, True
            SMI_waxs.masks[4][858, 5], SMI_waxs.masks[4][859, 5]=True, True
            SMI_waxs.masks[1][414, 6], SMI_waxs.masks[4][414, 6], =True, True
            SMI_waxs.masks[1][394, 138], SMI_waxs.masks[4][394, 138]=True, True
            SMI_waxs.masks[1][364, 41], SMI_waxs.masks[1][365, 41]=True, True
            SMI_waxs.masks[4][364, 41], SMI_waxs.masks[4][365, 41]=True, True
            SMI_waxs.masks[1][364, 96], SMI_waxs.masks[1][365, 96]=True, True
            SMI_waxs.masks[4][364, 96], SMI_waxs.masks[4][365, 96]=True, True
            SMI_waxs.masks[1][304, 96], SMI_waxs.masks[1][305, 96], SMI_waxs.masks[1][304, 97], SMI_waxs.masks[1][305, 97]=True, True, True, True
            SMI_waxs.masks[4][304, 96], SMI_waxs.masks[4][305, 96], SMI_waxs.masks[4][304, 97], SMI_waxs.masks[4][305, 97]=True, True, True, True
            SMI_waxs.masks[1][988, 188:194], SMI_waxs.masks[4][988, 188:194]= True, True
            SMI_waxs.masks[1][:, 1], SMI_waxs.masks[4][:, 1]= True, True
            SMI_waxs.masks[1][473, 20], SMI_waxs.masks[4][473, 20]= True, True
            SMI_waxs.masks[1][98, 5], SMI_waxs.masks[4][98, 5]= True, True
            SMI_waxs.masks[1][141, 111], SMI_waxs.masks[4][141, 111]= True, True
            SMI_waxs.masks[1][240:300, 0:50], SMI_waxs.masks[4][240:300, 0:50]= True, True
            SMI_waxs.masks[1][300:425, 125:], SMI_waxs.masks[4][300:425, 125:]= True, True
            SMI_waxs.masks[1][181, 97], SMI_waxs.masks[1][182, 97], SMI_waxs.masks[1][181, 98], SMI_waxs.masks[1][182, 98]=True, True, True, True
            SMI_waxs.masks[4][181, 97], SMI_waxs.masks[4][182, 97], SMI_waxs.masks[4][181, 98], SMI_waxs.masks[4][182, 98]=True, True, True, True
            for i, (img, mask) in enumerate(zip(SMI_waxs.imgs, SMI_waxs.masks)):
                if i%3==0:
                    SMI_waxs.imgs[i] = SMI_waxs.imgs[i]/flatfield[:, :195]+1E-7
                    SMI_waxs.masks[i][np.where(flatfield[:, :195]<1)] = True

                if i%3==1:
                    SMI_waxs.imgs[i] = SMI_waxs.imgs[i]/flatfield[:, 211:406]+1E-7
                    SMI_waxs.masks[i][np.where(flatfield[:, 211:406]<1)] = True

                if i%3==2:
                    SMI_waxs.imgs[i] = SMI_waxs.imgs[i]/flatfield[:, -195:]+1E-7
                    SMI_waxs.masks[i][np.where(flatfield[:, -195:]<1)] = True

            # Stitch detector images
            SMI_waxs.stitching_data(interp_factor=3, flag_scale=False)
            
            SMI_waxs.caking()
            
            filename_wa0_list.append(dat[0])
            recip_list.append(SMI_waxs.img_st)
            recip_extents.append([SMI_waxs.qp[0], SMI_waxs.qp[-1], SMI_waxs.qz[0], SMI_waxs.qz[-1]])
            
            caked_list.append(SMI_waxs.cake)
            caked_extents.append([SMI_waxs.q_cake[0], SMI_waxs.q_cake[-1], SMI_waxs.chi_cake[0], SMI_waxs.chi_cake[-1]])
            
    return filename_wa0_list, recip_list, recip_extents, caked_list, caked_extents

def waxs_SMI_numpy_loading_wa0wa20(path, filename_sublists):
    """
    Function adapted from Guillames SMI notebooks to process raw waxs tiffs:
    For waxs arc angles 0 and 20 degrees
    
    Inputs:
        path (pathlib.Path): 
        filename_sublists (list): 
    
    Returns: 
        filename_wa0_list (list): list of filenames loaded
        recip_list (list): list of loaded numpy data arrays (in qx vs qy space)
        recip_extents (list): list of qx qy extents: [qx_min, qx_max, qy_min, qy_max]
        caked_list (list): list of loaded numpy data arrays (in q vs chi space)
        caked_extents (list): list of q chi extents: [q_min, q_max, chi_min, chi_max]
    """
    
    # Set geometry
    geometry = 'Transmission'
    bs_kind = 'pindiode'
    incident_angle = np.deg2rad(0)

    # WAXS
    detector_waxs = 'Pilatus900kw'
    sdd_waxs = 280
    center_waxs = [97, 1255.9]
    bs_pos_waxs = [[97, 1070], [0, 0], [0, 0]]  # setings for waxs arc of zero degrees
    
    flatPath = pathlib.Path('/nsls2/data/smi/legacy/results/data/2024_3/314483_Freychet_08/900KW')    
    flatfield = np.rot90(fabio.open(flatPath.joinpath('GF_GF_flatfield_Sedge_2450uhighg1600_Y2_06_2477.00eV_wa20deg_id807229_000000_WAXS.tif')).data, 1)
    
    filename_wa0_list = []
    recip_list = []
    recip_extents = []
    caked_list = []
    caked_extents = []
    for dat in tqdm(filename_sublists, desc='Processing tiffs'):
            # Set awaxsw arc angles
            waxs_angle = [np.deg2rad(-0.06), np.deg2rad(19.7-0.06)]
        
            # Extract and set energy from filename
            idx = dat[0].find('eV')
            energy = 0.001*float(dat[0][idx-7:idx])
            wav = 1E-10 * (12.398/energy)

            # Define SMI loader object
            SMI_waxs = SMI_beamline.SMI_geometry(geometry = geometry,
                                                 detector = detector_waxs,
                                                 sdd = sdd_waxs,
                                                 wav = wav,
                                                 alphai = 0,
                                                 center = center_waxs,
                                                 bs_pos = bs_pos_waxs,
                                                 det_angles = waxs_angle,
                                                 bs_kind = None)


            # Load data, be aware of load order
            SMI_waxs.open_data(path, dat[::1], optional_mask='tender')

            # Masking code from Guillame
            SMI_waxs.masks[1][1180:, 100:108] = True
            SMI_waxs.masks[1][1254, 47], SMI_waxs.masks[1][1255, 47]=True, True
            SMI_waxs.masks[4][1254, 47], SMI_waxs.masks[4][1255, 47]=True, True
            SMI_waxs.masks[1][979:1050, 0:100], SMI_waxs.masks[4][979:1050, 0:100]=True, True
            SMI_waxs.masks[1][967, 67], SMI_waxs.masks[4][967, 67]=True, True
            SMI_waxs.masks[1][490:555, 100:], SMI_waxs.masks[4][490:555, 100:]=True, True
            SMI_waxs.masks[1][1231, 174], SMI_waxs.masks[1][1232, 174]=True, True
            SMI_waxs.masks[4][1231, 174], SMI_waxs.masks[4][1232, 174]=True, True
            SMI_waxs.masks[1][1414, 179], SMI_waxs.masks[1][1415, 179]=True, True
            SMI_waxs.masks[4][1414, 179], SMI_waxs.masks[4][1415, 179]=True, True
            SMI_waxs.masks[1][858, 5], SMI_waxs.masks[1][859, 5]=True, True
            SMI_waxs.masks[4][858, 5], SMI_waxs.masks[4][859, 5]=True, True
            SMI_waxs.masks[1][414, 6], SMI_waxs.masks[4][414, 6], =True, True
            SMI_waxs.masks[1][394, 138], SMI_waxs.masks[4][394, 138]=True, True
            SMI_waxs.masks[1][364, 41], SMI_waxs.masks[1][365, 41]=True, True
            SMI_waxs.masks[4][364, 41], SMI_waxs.masks[4][365, 41]=True, True
            SMI_waxs.masks[1][364, 96], SMI_waxs.masks[1][365, 96]=True, True
            SMI_waxs.masks[4][364, 96], SMI_waxs.masks[4][365, 96]=True, True
            SMI_waxs.masks[1][304, 96], SMI_waxs.masks[1][305, 96], SMI_waxs.masks[1][304, 97], SMI_waxs.masks[1][305, 97]=True, True, True, True
            SMI_waxs.masks[4][304, 96], SMI_waxs.masks[4][305, 96], SMI_waxs.masks[4][304, 97], SMI_waxs.masks[4][305, 97]=True, True, True, True
            SMI_waxs.masks[1][988, 188:194], SMI_waxs.masks[4][988, 188:194]= True, True
            SMI_waxs.masks[1][:, 1], SMI_waxs.masks[4][:, 1]= True, True
            SMI_waxs.masks[1][473, 20], SMI_waxs.masks[4][473, 20]= True, True
            SMI_waxs.masks[1][98, 5], SMI_waxs.masks[4][98, 5]= True, True
            SMI_waxs.masks[1][141, 111], SMI_waxs.masks[4][141, 111]= True, True
            SMI_waxs.masks[1][240:300, 0:50], SMI_waxs.masks[4][240:300, 0:50]= True, True
            SMI_waxs.masks[1][300:425, 125:], SMI_waxs.masks[4][300:425, 125:]= True, True
            SMI_waxs.masks[1][181, 97], SMI_waxs.masks[1][182, 97], SMI_waxs.masks[1][181, 98], SMI_waxs.masks[1][182, 98]=True, True, True, True
            SMI_waxs.masks[4][181, 97], SMI_waxs.masks[4][182, 97], SMI_waxs.masks[4][181, 98], SMI_waxs.masks[4][182, 98]=True, True, True, True
            for i, (img, mask) in enumerate(zip(SMI_waxs.imgs, SMI_waxs.masks)):
                if i%3==0:
                    SMI_waxs.imgs[i] = SMI_waxs.imgs[i]/flatfield[:, :195]+1E-7
                    SMI_waxs.masks[i][np.where(flatfield[:, :195]<1)] = True

                if i%3==1:
                    SMI_waxs.imgs[i] = SMI_waxs.imgs[i]/flatfield[:, 211:406]+1E-7
                    SMI_waxs.masks[i][np.where(flatfield[:, 211:406]<1)] = True

                if i%3==2:
                    SMI_waxs.imgs[i] = SMI_waxs.imgs[i]/flatfield[:, -195:]+1E-7
                    SMI_waxs.masks[i][np.where(flatfield[:, -195:]<1)] = True

            # Stitch detector images
            SMI_waxs.stitching_data(interp_factor=3, flag_scale=False)
            
            SMI_waxs.caking()
            
            filename_wa0_list.append(dat[0])
            recip_list.append(SMI_waxs.img_st)
            recip_extents.append([SMI_waxs.qp[0], SMI_waxs.qp[-1], SMI_waxs.qz[0], SMI_waxs.qz[-1]])
            
            caked_list.append(SMI_waxs.cake)
            caked_extents.append([SMI_waxs.q_cake[0], SMI_waxs.q_cake[-1], SMI_waxs.chi_cake[0], SMI_waxs.chi_cake[-1]])
            
    return filename_wa0_list, recip_list, recip_extents, caked_list, caked_extents

## Load data & save zarrs

In [ ]:
all_waxs = set(waxsPath.glob('*.tif'))
sample_names = sorted(set([f.name[3:f.name.find('_sdd')] for f in all_waxs]))[:]
sample_names

In [ ]:
# sample_names = ['TEGDME_neat', 'Li2S_TEGDME_reredo']
sample_names = ['Li2S8_static']

In [ ]:
def remove_duplicate_energies(filename_list):
    """Just a helper function to remove duplicate energy files if there are any later on"""
    
    file_energies = []
    kept_files = []

    for filename in filename_list:
        file_energy = filename[:filename.find('eV')].split('_')[-1]
        if file_energy in file_energies:
            pass
        else:
            file_energies.append(file_energy)
            kept_files.append(filename)

    return kept_files

In [ ]:
for sample_name in tqdm(sample_names[:], desc='Samples'):
    # Select files for a given sample and rotation
    all_paths = sorted(waxsPath.glob(f'*TC_{sample_name}_sdd*'))

    # Select samples
    filename_list = [f.name for f in sorted(all_paths)]
    
    # We need to check that the number of energies for the 2 detector positions is consistent
    num_energies = 63  # adjust to total number of energies for this scan
    wa1_filename_list = [fname for fname in filename_list if 'wa0' in fname]  # select waxs arc angle 1
    wa2_filename_list = [fname for fname in filename_list if 'wa20' in fname]  # select waxs arc angle 2
    if len(wa1_filename_list) == num_energies:
        pass
    else:
        wa1_filename_list = remove_duplicate_energies(wa1_filename_list)

    if len(wa2_filename_list) == num_energies:
        pass
    else:
        wa2_filename_list = remove_duplicate_energies(wa2_filename_list)

    filename_list =  sorted(wa1_filename_list + wa2_filename_list)

    # Make sublists to stitch two waxs positions together
    group_size = 2
    filename_sublists = [filename_list[i:i + group_size] for i in range(0, len(filename_list), group_size)]

    # Run SMI loading code, make sure to use the correct function! (this produces some fabio and divide by zero errors)
    names_list, recip_list, recip_extents, caked_list, caked_extents = waxs_SMI_numpy_loading_wa0wa20(waxsPath, filename_sublists)
    # names_list, recip_list, recip_extents, caked_list, caked_extents = waxs_SMI_numpy_loading_wa7wa27(waxsPath, filename_sublists)

    # Construct xarrays with full values along detector dimensions and the energy dimension
    # They contain sample name and theta value as well, as single values to be concatenated in later steps
    recip_DA_rows = []
    caked_DA_rows = []
    zipped_lists = zip(names_list, recip_list, recip_extents, caked_list, caked_extents)
    for filename, recip_arr, recip_extent, caked_arr, caked_extent in zipped_lists:

        attr_dict = {}
        attr_dict['filename'] = filename
        energy = float(filename[:filename.find('eV')].split('_')[-1])

        recip_DA = xr.DataArray(data = recip_arr, 
                                dims = ['pix_y', 'pix_x'],
                                attrs = attr_dict)
        recip_DA = recip_DA.assign_coords({
            'pix_x': recip_DA.pix_x.data,
            'pix_y': recip_DA.pix_y.data,
            'q_x': ('pix_x', np.linspace(recip_extent[0], recip_extent[1], len(recip_DA.pix_x.data))),
            'q_y': ('pix_y', np.linspace(recip_extent[3], recip_extent[2], len(recip_DA.pix_y.data)))
        })
        recip_DA = recip_DA.expand_dims({
            'energy': [energy],
            'sample_name': [sample_name],
            'theta': [90]
        })
        recip_DA_rows.append(recip_DA)

        caked_DA = xr.DataArray(data = caked_arr, 
                                dims = ['index_y', 'index_x'],
                                attrs = attr_dict)
        caked_DA = caked_DA.assign_coords({
            'index_x': caked_DA.index_x.data,
            'index_y': caked_DA.index_y.data,
            'q_r': ('index_x', np.linspace(caked_extent[0], caked_extent[1], len(caked_DA.index_x.data))),
            'chi': ('index_y', np.linspace(caked_extent[3], caked_extent[2], len(caked_DA.index_y.data)))
        }) 
        caked_DA = caked_DA.expand_dims({
            'energy': [energy],
            'sample_name': [sample_name],
            'theta': [90]
        })
        caked_DA_rows.append(caked_DA)

    recip_DA = xr.concat(recip_DA_rows, 'energy')
    caked_DA = xr.concat(caked_DA_rows, 'energy')

    # Save sample zarr, load later to concatenate full zarr
    # sampleZarrsPath = propPath.joinpath('processed_data/zarrs/waxs_last_morning')
    sampleZarrsPath = propPath.joinpath('processed_data/zarrs/waxs_Li2S8_static_solution_zarrs_v1')
    sampleZarrsPath.mkdir(exist_ok=True)

    recip_samp_zarr_name = 'recip_'+recip_DA.sample_name.values[0]+'_'+str(int(recip_DA.theta.values[0]))+'deg.zarr'
    recip_DS = recip_DA.to_dataset(name='flatfield_corr')
    recip_DS.to_zarr(sampleZarrsPath.joinpath(recip_samp_zarr_name), mode='w')

    caked_samp_zarr_name = 'caked_'+caked_DA.sample_name.values[0]+'_'+str(int(caked_DA.theta.values[0]))+'deg.zarr'
    caked_DS = caked_DA.to_dataset(name='flatfield_corr')
    caked_DS.to_zarr(sampleZarrsPath.joinpath(caked_samp_zarr_name), mode='w')

#### Now switch to to plotting notebook!